<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_NG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PBMC_NG_2 dataset: Processes the BUG files into files prepared for use in R**

This notebook processes the output from the fastq file processing for this dataset. The data produced here is pre-generated and downloaded by the figure generation code. The purpose of this processing step is to prepare the data for figure generation, by filtering the data and producing downsampled datasets in addition to the original one. 

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process the data
4. Generate statistics for the dataset

The data used in this processing step is produced by the following notebook:

https://github.com/pachterlab/GRNP_2020/blob/master/notebooks/FASTQ_processing/ProcessPBMC_NG_2.ipynb


**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 1766 (delta 26), reused 10 (delta 1), pack-reused 1718
Receiving objects: 100% (1766/1766), 9.44 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (1211/1211), done.


In [2]:
#download BUG data from Zenodo
!mkdir data
!cd data && wget https://zenodo.org/record/4661238/files/PBMC_NG_2.zip?download=1 && unzip 'PBMC_NG_2.zip?download=1' && rm 'PBMC_NG_2.zip?download=1'

--2021-04-03 15:40:47--  https://zenodo.org/record/4661238/files/PBMC_NG_2.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477401560 (455M) [application/octet-stream]
Saving to: ‘PBMC_NG_2.zip?download=1’

PBMC_NG_2.zip?downl 100%[===================>] 455.29M  9.75MB/s    in 49s     

2021-04-03 15:41:37 (9.24 MB/s) - ‘PBMC_NG_2.zip?download=1’ saved [477401560/477401560]

Archive:  PBMC_NG_2.zip?download=1
   creating: PBMC_NG_2/
   creating: PBMC_NG_2/bus_output/
  inflating: PBMC_NG_2/bus_output/bug.txt  
  inflating: PBMC_NG_2/bus_output/coll.genes.txt  
  inflating: PBMC_NG_2/bus_output/transcripts_to_genes.txt  


In [3]:
#Check that download worked
!cd data && ls -l && cd PBMC_NG_2/bus_output && ls -l

total 4
drwxr-xr-x 3 root root 4096 Jul  1  2020 PBMC_NG_2
total 2210072
-rw-r--r-- 1 root root 2254025859 Jul  1  2020 bug.txt
-rw-r--r-- 1 root root     738211 Jun 30  2020 coll.genes.txt
-rw-r--r-- 1 root root    8335642 Jun 30  2020 transcripts_to_genes.txt


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


In [5]:
#install the R packages
%%R
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")
install.packages("stringr")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process the data**

Here we discard multimapped UMIs and all UMIs belonging to cells with fewer than 200 UMIs. We also precalculate gene expression, fraction of single-copy molecules etc. and save as stats (statistics). These can later be used when generating figures. We also generate down-sampled BUGs.

In [6]:
#create output directory
!mkdir figureData

In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [8]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "BUGProcessingHelpers.R"))
createStandardBugsData(paste0(dataPath,"PBMC_NG_2/"), "PBMC_NG_2", c(0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1))



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘qdapTools’


R[write to console]: The following object is masked from ‘package:dplyr’:

    id




[1] "Generating data for PBMC_NG_2"
[1] "Reading BUG from data/PBMC_NG_2/ ..."
[1] "Filtering multi-mapped reads..."
[1] "Fraction multi-mapped reads: 0.168672376634181"
[1] "Converting genes..."
[1] "Done"
[1] "Down-sampling in total 7 bugs:"
[1] "7: Down-sampling to 1"
[1] "saving BUG..."
[1] "creating stats..."
[1] "cpm normalizing..."
[1] "6: Down-sampling to 0.8"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."
[1] "cpm normalizing..."
[1] "5: Down-sampling to 0.6"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."
[1] "cpm normalizing..."
[1] "4: Down-sampling to 0.4"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."
[1] "cpm normalizing..."
[1] "3: Down-sampling to 0.2"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


**4. Generate statistics for the dataset**

Here we create a file with various statistics for the dataset, which is used for generating table S3. It also contains some additional information about the dataset. Generation of this file may take several hours.

In [9]:
%%R
source(paste0(sourcePath, "GenBugSummary.R"))
genBugSummary("PBMC_NG_2", "FGF23", "RPS10", 10)

[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 10

In [10]:
!cd figureData/PBMC_NG_2 && ls -l && cat ds_summary.txt

total 1798252
-rw-r--r-- 1 root root 406579011 Apr  3 15:57 Bug_100.RData
-rw-r--r-- 1 root root 125842777 Apr  3 16:23 Bug_10.RData
-rw-r--r-- 1 root root 204740969 Apr  3 16:21 Bug_20.RData
-rw-r--r-- 1 root root 297249636 Apr  3 16:17 Bug_40.RData
-rw-r--r-- 1 root root  70841545 Apr  3 16:25 Bug_5.RData
-rw-r--r-- 1 root root 350319700 Apr  3 16:11 Bug_60.RData
-rw-r--r-- 1 root root 383599007 Apr  3 16:04 Bug_80.RData
-rw-r--r-- 1 root root       983 Apr  3 21:22 ds_summary.txt
-rw-r--r-- 1 root root   2212656 Apr  3 16:25 Stats.RData
Dataset: PBMC_NG_2

totUMIs: 40658153
totCells: 5364
totCounts: 155012999
countsPerUMI: 3.81259323314564
UMIsPerCell: 7579.8197240865
countsPerCell: 28898.769388516
totFracOnes: 0.247950982918481
FracMolWithUMIDistToNeighborH: 64, 690, 1177, 69, 0, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborL: 71, 709, 1108, 107, 5, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborHFrac: 0.032, 0.345, 0.5885, 0.0345, 0, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborLFrac: 0.0355